In [ ]:
import os, sys, time, json, re, io
import base64
import dotenv
import requests
import subprocess
import pandas as pd
from datasets import load_dataset

from IPython.display import display, HTML
from PIL import Image 

dotenv.load_dotenv("../.env")
sys.path.append("..")
from evalap.utils import log_and_raise_for_status

EVALAP_API_URL = "http://localhost:8000/v1"
#EVALAP_API_URL = "http://localhost:8000/v1"
EVALAP_API_KEY = os.getenv("EVALAP_API_KEY") 
ALBERT_API_URL = "https://albert.api.etalab.gouv.fr/v1"
ALBERT_API_KEY = os.getenv("ALBERT_API_KEY")

evalap_headers = {"Authorization": f"Bearer {EVALAP_API_KEY}"}
albert_headers = {"Authorization": f"Bearer {ALBERT_API_KEY}"}

In [ ]:
# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("datalab-to/marker_benchmark")

In [ ]:
ds

In [ ]:
# What a row contains 

for key, value in ds["train"][1261].items():
    print(key, type(value))

In [ ]:
# Example of the first block content (gt_blocks)

jsb = json.loads(ds["train"][0]["gt_blocks"])

for block in jsb:
    print(list(block))
    print(block)
    break

In [ ]:
def display_dataset_row(row):
    """
    Display a dataset row nicely in a Jupyter notebook.
    
    Args:
    - row: dictionary containing elements like 'gt_blocks', 'img', etc.
    """
    for key, value in row.items():
        if isinstance(value, Image.Image):  # Use Image.Image to check for PIL images
            print(f"{key}: Image")
            display(value)
        elif key == "gt_blocks":
            blocks = json.loads(value)
            print(f'{len(blocks)} blocks. Block types: {[x.get("block_type") for x in blocks]}')
            #for block in blocks:
            #    print(block["html"])
        elif isinstance(value, str):
            print(f"{key}: {value}")
        elif isinstance(value, list):
            print(f"{key}: {value}")
        elif isinstance(value, int):
            print(f"{key}: {value}")
        elif isinstance(value, bytes):
            print(f"{key}: {len(value)} bytes")
        else:
            print(f"{key}: {type(value)} (unable to display)")

def image_to_base64(pil_image):
    """
    Convert a PIL image to a base64-encoded PNG bytes string.
    """
    with io.BytesIO() as buffer:
        pil_image.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")

def extract_code(text: str):
    """Return the last code block found"""
    # Find all blocks of code wrapped in triple backticks
    matches = re.findall(r"```(?:\w+)?\n(.*?)```", text, re.DOTALL)
    if matches:
        # Return the last code block found
        code = matches[-1]
        # Remove any leading or trailing whitespace
        code = code.strip()
        return code

    return text.strip()


def pandoc(content, input_format="html", output_format="markdown"):
    """
    Convert HTML to Markdown using Pandoc

    Parameters:
    - html_content: The HTML string to convert
    - output_format: The specific Markdown flavor (e.g., "markdown", "markdown_strict", "gfm" for GitHub-flavored Markdown)

    Returns:
    - Markdown text
    """
    # Create a process that runs pandoc
    process = subprocess.Popen(
        ['pandoc', '-f', input_format, '-t', output_format],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    # Send the HTML content to pandoc and get the output
    new_content, error = process.communicate(input=content)
    if process.returncode != 0:
        raise Exception(f"Pandoc conversion failed: {error}")

    return new_content


In [ ]:
display_dataset_row(ds["train"][0])

In [ ]:
system_prompt = """You are tasked with generating a JSON representation from the analysis (OCR) of a given image. 
Your goal is to create a JSON object that has the content of the image extracted in a structured format. 

The JSON should be structured to represent the content of the image in a way that corresponds to standard markdown primitives. Here's how to approach this task:

The JSON should contain a list of blocks, where each block represents a distinct element in the image, such as headers, paragraphs, or tables.
Here is a an exemple of the json schema wanted: 

Schema:
```json
[
 {
   "type": "string (e.g  Text, Table, Code, SectionHeader, Figure, Equation, Handwriting, PageFooter, PageHeader, Picture, TableOfContents etc)",
   "text": "string (mardown formated text)"
 },
 ...
]
```

Example:
```json
[
 {
    "type": "Header", "text": "## I am a level 2 header"
 },
 {
   "type": "Paragraph", "text": "I am a **paragraph**"
 }
]
````

Follow these guidelines when creating the JSON:

1. The main structure should be a list of blocks. Each block are object containing a `type` and a `text field`.
2. Each block is an object containing a `type` and a `text field`. They should correspond to a standard markdown primitive (e.g., Header, Paragraph, Table).
3. Identify headers based on font size, weight, or positioning. These should be represented as "Header" blocks.
4. Group continuous lines of text into "Paragraph" blocks.
5. Identify tabular data and represent it as "Table" blocks. Only create table blocks for actual tabular data, not for text formatting.
6. Do not create separate blocks for inline formatting (bold, italic) or URLs. Keep these within the relevant "Paragraph" block.
7. If you encounter lists, represent them as "List" blocks, with nested items if applicable.
8. For images or diagrams, use an "Image" block and include any available descriptive text.

Remember, the goal is to create a structured representation of the image content that could be easily converted to markdown or used for further processing. Focus on the main structural elements and avoid over-complicating the JSON with minor formatting details.

Do not explain your answer. Just answer with the JSON result directly.
"""

In [ ]:
# Albert OCR request
# @DEBUG: do not work yet

url = "https://albert.api.etalab.gouv.fr/v1/ocr-beta"
model = "mistralai/Mistral-Small-3.1-24B-Instruct-2503"
payload = {
    "model": model,
    "prompt": system_prompt,
    #"file": "data:image/png;base64," + image_to_base64(ds["train"][0]["img"]),
}

ds["train"][0]["img"].save('tmp-img-marker-0.pdf', "PDF", resolution=300)
files = {
    'file': open("tmp-img-marker-0.pdf", "rb")
}

# Send the request
response = requests.post(url, headers=albert_headers, data=payload, files=files)
log_and_raise_for_status(response, "LLM API error")
result = response.json()
result

In [ ]:
url = "https://albert.api.etalab.gouv.fr/v1/chat/completions"
model = "mistralai/Mistral-Small-3.1-24B-Instruct-2503"
messages = [
    #{
    #    "role": "system",
    #    "content": "system_prompt
    #},
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": system_prompt
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": "data:image/png;base64," + image_to_base64(ds["train"][0]["img"]),
                }
            }
        ]
    }
]

payload = {
    "model": model,
    "messages": messages,
    "temperature": 0.2
}

# Send the request
response = requests.post(url, headers=albert_headers, json=payload)
log_and_raise_for_status(response, "LLM API error")
result = response.json()

In [ ]:
json.loads(extract_code(result["choices"][0]["message"]["content"]))

### Push dataset on EVALAP

1. add `json_output_true` column in the dataset
2. create the dataset (empty)
3. stream the parquet data

In [ ]:
# Create the `json_output_true`

def create_json_output_true(row):
    blocks = json.loads(row["gt_blocks"])
    v = []
    texts = []
    for block in blocks:
        text = pandoc(block["html"]).strip()
        v.append({
            'bbox': block["bbox"], 
            'polygon': block["polygon"], 
            'type': block["block_type"],
            'text': text,
        })
        texts.append(text)
    markdown = "\n\n".join(texts)
    return {"json_output_true": v, "markdown_true": markdown}

ds["train"] = ds["train"].map(create_json_output_true)
ds["train"] = ds["train"].remove_columns(["gt_blocks"])

In [ ]:
# Try to save the dataset as a parquet file (its around 700 Mo)
ds["train"].to_parquet("dataset_test.parquet", compression="gzip") 

In [ ]:
import pyarrow.parquet as pq

pf = pq.ParquetFile("dataset_test.parquet")
num_rows = pf.metadata.num_rows
column_names = pf.schema_arrow.names

print("num rows", num_rows)
print("column names", column_names)

In [ ]:
i = 0
for batch in pf.iter_batches(batch_size=1):
    df = batch.to_pandas()
    display(df.head())
    # Open the image using Pillow
    image = Image.open(io.BytesIO(df["img"].iloc[0]["bytes"]))
    print(image.format)
    i += 1
    if i >= 2:
        break

In [ ]:
# Create the dataset
dataset = {"name": "OCR_marker_benchmark", "readme": "Marker benchmark. See https://github.com/VikParuchuri/marker"
           , "default_metric" : "ocr_v1"
           , "df": "{}"}

response = requests.post(f'{EVALAP_API_URL}/dataset', json=dataset, headers=evalap_headers)
dataset = response.json()
dataset

In [ ]:
# Function to stream data in chunks
def stream_data(buffer, chunk_size=1024*1024):
    buffer.seek(0)
    while True:
        chunk = buffer.read(chunk_size)
        if not chunk:
            break
        yield chunk


# Create an in-memory buffer
buffer = io.BytesIO()

# Write directly to parquet without converting to pandas first
ds['train'].to_parquet(buffer, compression="gzip")

# Upload the data using a streaming POST request
headers = {'Content-Type': 'application/octet-stream'} | evalap_headers
response = requests.post(f'{EVALAP_API_URL}/dataset/{dataset["id"]}/upload_parquet', data=stream_data(buffer, chunk_size=262144), headers=headers)

# Check the response
if response.ok:
    print("Upload successful!")
    print(response.json())
else:
    print("Upload failed:", response.status_code, response.text)

In [ ]:
# Patch the dataset to add a dolumn map to indicate where is **output_true**
# --

# Define the data to be patched
data = {
    "columns_map": {
        "output_true": "markdown_true",
    }
}

# Make the PATCH request
response = requests.patch(f'{EVALAP_API_URL}/dataset/{dataset["id"]}', headers=evalap_headers, data=json.dumps(data))

# Print the response
print(response.status_code)
print(response.json())